In [1]:
#this notebook reads the experimental and simulation data from a defined directory
#then the chi2 and native popuations for a given set of parameters are computed
#ultimately the results published in this study are obtained

In [2]:
import bussilab 
import scipy
from scipy.optimize import minimize
import cudamat as cm
import numpy as np
import matplotlib.pyplot as plt
import re
kBT=0.6 #kBT in kcal/mol
np.random.seed(1995)
import os

In [3]:
#function takes list of file names and concatenated them
def concatenate_simulation_data(files):
    result=None
    for t in files:
        if result is None:
            result=np.load(t)
            output=+result
        else:
            result=np.load(t)
            output=np.concatenate((output,result))

    return output

def read_TLs(files):
    corr=[]
    for file in files:
        with open(file, "r") as f:
            for line in f:
                nums=line.split()
                if nums:
                    corr.append(np.array([float(i) for i in nums]))
    return np.array(corr)

In [4]:
#for the TLs the bias is computed for the respective other forcefield as well both times
def calculate_TL_bias(Sequence,key,prune,skip,trajGAGA,FFprefactorsGAGA,trajUUCG,FFprefactorsUUCG):
    
    directories=["nh-n_0.5_nh-o_0.5_oh-bo-nbO_-0.5_dumping", 
                 "nh-n_1.0_nh-o_0.0_oh-bo-nbO_-0.5_dumping_bias-from-half-half",
                 "nh-n_0.5_nh-o_0.5_oh-bo-nbO_-0.5_dumping_bias-from-one-zero",
                 "nh-n_1.0_nh-o_0.0_oh-bo-nbO_-0.5_dumping"
                 ]

    collection_weights=[]
    for index,d in enumerate(directories):
        Metadweight=[]
        with open("./data_loaded/%s/%s/weights.rep0" %(Sequence,d) ,"r") as fp:
            for line in fp:
                Metadweight.append(float(line))
        collection_weights.append(np.array(Metadweight))

    MetadPot1=np.concatenate((collection_weights[0],collection_weights[1]))
    MetadPot2=np.concatenate((collection_weights[2],collection_weights[3]))

    MetadPot=np.c_[ MetadPot1,MetadPot2 ] 
    MetadPot=kBT*np.log(MetadPot)
    
    if prune==True:
        MetadPot=MetadPot[::skip,:]
    
    if Sequence=='GAGA':
        if key == 'reference':
            bias=np.matmul(trajGAGA,np.array(FFprefactorsGAGA).T)+MetadPot
        if key == 'proposed':
            bias=np.matmul(trajGAGA,(FFprefactorsGAGA-FFprefactorsGAGA[1]).T)+MetadPot
        
    if Sequence=='UUCG':
        if key == 'reference':
            bias=np.matmul(trajUUCG,np.array(FFprefactorsUUCG).T)+MetadPot
        if key == 'proposed':
            bias=np.matmul(trajUUCG,(FFprefactorsUUCG-FFprefactorsUUCG[1]).T)+MetadPot
        

    del MetadPot1
    del MetadPot2
    del MetadPot
        
    return bias   

In [5]:
def activate_cuda(traj,weights, USE_CUDA):

    if USE_CUDA:
        cm.init(max_ones=traj.shape[0])
        cm_traj=cm.CUDAMatrix(traj.T)
        cm_weights=cm.CUDAMatrix(np.reshape(weights,(1,-1)))
        return cm_traj, cm_weights
    else:
        return traj,weights

def compute_newweights(par,cm_traj,cm_weights,USE_CUDA):
    if USE_CUDA:
        cm_par=cm.CUDAMatrix(np.reshape(par,(1,-1)))
        cm_correction=cm.dot(cm_par,cm_traj)
        
        m=float(cm_correction.min(axis=1).asarray()[0,0])
        cm_correction.subtract(m)
        cm_correction.mult(-1.0)
        newweights=cm.exp(cm_correction)
        newweights.mult_by_row(cm_weights)
        newweights.mult(1.0/(cm.sum(newweights,axis=1).asarray()[0,0]))
        return newweights 
    else:
        correction=np.matmul(cm_traj,par)
        correction-=np.min(correction)
        newweights=np.exp(-correction)*cm_weights
        return newweights/np.sum(newweights)
    
def function(par,obs,newweights,traj,cm_traj,USE_CUDA):
    gradient=True
    if USE_CUDA:
        if(isinstance(obs,cm.CUDAMatrix)):
            cm_obs=obs
        else:
            cm_obs=cm.CUDAMatrix(obs)
        if(isinstance(newweights,cm.CUDAMatrix)):
            cm_newweights=newweights
        else:
            cm_newweights=cm.CUDAMatrix(np.reshape(newweights,(1,-1)))
        av_obs=np.reshape(np.array(cm.dot(cm_newweights,cm_obs).asarray()),(-1))
    else:
        av_obs=np.reshape(np.array(np.matmul(newweights,obs)),(-1))
    
    if not gradient:
        return av_obs
    
    dav=np.zeros((len(par),obs.shape[1]))
    
    if USE_CUDA:
        cm_traj_weighted=cm.empty((traj.shape[1],traj.shape[0]))
        cm_traj_weighted.assign(cm_traj)
        cm_traj_weighted.mult_by_row(cm_newweights)
        av_traj=np.reshape(cm_traj_weighted.sum(axis=1).asarray(),(-1))
        dav=-cm.dot(cm_traj_weighted,cm_obs).asarray()
    else:
        weighted_traj=np.copy(traj)
        for ipar in range(len(par)):
            weighted_traj[:,ipar]=newweights*traj[:,ipar]
        av_traj=np.matmul(newweights,traj)
        for ipar in range(len(par)):
            for iobs in range(obs.shape[1]):
                dav[ipar,iobs]=-np.sum(weighted_traj[:,ipar]*obs.T[iobs,:])

    for ipar in range(len(par)):
        for iobs in range(obs.shape[1]):
            dav[ipar,iobs]+=av_obs[iobs]*av_traj[ipar]
            
    return av_obs,dav

In [6]:
def _logsum(ns):
    max = np.max(ns)
    ds = ns - max
    sumOfExp = np.exp(ds).sum()
    return max + np.log(sumOfExp)

In [7]:
def _softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [8]:
#load training set data
print("Load experimental data")
noe_exp=np.loadtxt("./data_loaded/GACC/exp_noes",usecols=(2,3))
unoe_exp=np.loadtxt("./data_loaded/GACC/exp_unobs-noes",usecols=(2,3))
sugar_exp=np.loadtxt("./data_loaded/GACC/exp_jcoupl-sugar",usecols=(1,2))
backbone1_exp=np.loadtxt("./data_loaded/GACC/exp_jcoupl-backbone-1",usecols=(1,2))
backbone2_exp=np.loadtxt("./data_loaded/GACC/exp_jcoupl-backbone-2",usecols=(1,2))
print(noe_exp.shape)
print(unoe_exp.shape)
print(sugar_exp.shape)
print(backbone1_exp.shape)
print(backbone2_exp.shape)
print('There are no experimental population data for the TLs. The population is enforced to be 0.5 in the cost function.')

#get the names of the different simulations and there forcefield prefactors
trajectoryNamesGACC=[]
trajectoryNamesGAGA=[]
trajectoryNamesUUCG=[]
FFprefactorsGACC=[]
FFprefactorsGAGA=[]
FFprefactorsUUCG=[]

with open("./data_loaded/coefficients.dat","r") as fp:
    for line in fp:
        if(line[0]=='#'):
            continue
        l=line.split()
        file=l[0]+".skip.npy"
        trajectoryNamesGACC.append(file)
        FFprefactorsGACC.append(np.array(np.array(l)[1:],dtype='float'))

with open("./data_loaded/coefficients_TLs.dat","r") as fp:
    for line in fp:
        if(line[0]=='#'):
            continue
        if line.strip():
            l=line.split()
            file=l[0]+".npy"
            if 'gaga' in line:
                FFprefactorsGAGA.append(np.array(l[1:],dtype='float'))
            if 'uucg' in line:
                FFprefactorsUUCG.append(np.array(l[1:],dtype='float'))

firstFF='nh-n_0.5_nh-o_0.5_oh-bo-nbO_-0.5_dumping/'
secondFF='nh-n_1.0_nh-o_0.0_oh-bo-nbO_-0.5_dumping/'
trajectoryNamesGAGA.append('./data_loaded/GAGA/'+firstFF+'gHBfix-parameters_state.rep0')
trajectoryNamesGAGA.append('./data_loaded/GAGA/'+secondFF+'gHBfix-parameters_state.rep0')
trajectoryNamesUUCG.append('./data_loaded/UUCG/'+firstFF+'gHBfix-parameters_state.rep0')
trajectoryNamesUUCG.append('./data_loaded/UUCG/'+secondFF+'gHBfix-parameters_state.rep0')

#-------------------------------------------------------------------------------
#load observables
print('There are 14 simulations for GACC.')

trajGACC=concatenate_simulation_data(['./data_loaded/GACC/' + x for x in trajectoryNamesGACC])
backbone1=concatenate_simulation_data(['./data_loaded/GACC/' + re.sub("HBfix-energy","jcop_backbone-1",x) for x in trajectoryNamesGACC])
backbone2=concatenate_simulation_data(['./data_loaded/GACC/' + re.sub("HBfix-energy","jcop_backbone-2",x) for x in trajectoryNamesGACC])
sugar=concatenate_simulation_data(['./data_loaded/GACC/' + re.sub("HBfix-energy","jcop_sugar",x) for x in trajectoryNamesGACC])
noe=concatenate_simulation_data(['./data_loaded/GACC/' + re.sub("HBfix-energy","noe",x) for x in trajectoryNamesGACC])
unoe=concatenate_simulation_data(['./data_loaded/GACC/' + re.sub("HBfix-energy","unoe",x) for x in trajectoryNamesGACC])

#apply forward models
sugar=9.67*np.cos(sugar*np.pi/180)**2 - 2.03*np.cos(sugar*np.pi/180)
backbone1=9.7*np.cos(backbone1*np.pi/180)**2 - 1.8*np.cos(backbone1*np.pi/180)
backbone2=15.3*np.cos(backbone2*np.pi/180)**2 - 6.1*np.cos(backbone2*np.pi/180)+1.6
noe=noe**-6
unoe=unoe**-6
print('There are 2 simulations per TL.')
trajGAGA=read_TLs([x for x in trajectoryNamesGAGA])
trajUUCG=read_TLs([x for x in trajectoryNamesUUCG])
populationGAGA=trajGAGA[:,-1].reshape(-1,1)
populationUUCG=trajUUCG[:,-1].reshape(-1,1)
trajGAGA=trajGAGA[:,:12]
trajUUCG=trajUUCG[:,:12]

#to obtain the same parameters as published set prune=False
prune=False
if prune==True:
    skip=5000
    print("Pruning data, skip")
    trajGACC=trajGACC[::skip,:]
    backbone1=backbone1[::skip,:]
    backbone2=backbone2[::skip,:]
    sugar=sugar[::skip,:]
    noe=noe[::skip,:]
    unoe=unoe[::skip,:]
    populationGAGA=populationGAGA[::skip,:]
    populationUUCG=populationUUCG[::skip,:]
    trajGAGA=trajGAGA[::skip,:]
    trajUUCG=trajUUCG[::skip,:]
else:
    skip=0

print("Check shape of arrays:")
print(trajGACC.shape)
print(backbone1.shape)
print(backbone2.shape)
print(sugar.shape)
print(noe.shape)
print(unoe.shape)
print(populationGAGA.shape)
print(populationUUCG.shape)
print(trajGAGA.shape)
print(trajUUCG.shape)

weightsGACC=np.exp(bussilab.wham.wham(np.matmul(trajGACC,np.transpose(FFprefactorsGACC-FFprefactorsGAGA[1])),threshold=1e-20,T=kBT).logW)
weightsGAGA=np.exp(bussilab.wham.wham(calculate_TL_bias('GAGA','proposed',prune,skip,trajGAGA,FFprefactorsGAGA,trajUUCG,FFprefactorsUUCG),threshold=1e-20,T=kBT).logW)
weightsUUCG=np.exp(bussilab.wham.wham(calculate_TL_bias('UUCG','proposed',prune,skip,trajGAGA,FFprefactorsGAGA,trajUUCG,FFprefactorsUUCG),threshold=1e-20,T=kBT).logW)

Load experimental data
(20, 2)
(285, 2)
(12, 2)
(8, 2)
(9, 2)
There are no experimental population data for the TLs. The population is enforced to be 0.5 in the cost function.
There are 14 simulations for GACC.
There are 2 simulations per TL.
Check shape of arrays:
(1400000, 12)
(1400000, 8)
(1400000, 9)
(1400000, 12)
(1400000, 20)
(1400000, 285)
(1000000, 1)
(1000000, 1)
(1000000, 12)
(1000000, 12)


In [9]:
#this is important for all TLs, they will be weighted the same as the GACC system in the training for consistency
TLweight=(noe_exp.shape[0]+unoe_exp.shape[0] + backbone1_exp.shape[0]+ backbone2_exp.shape[0]+sugar_exp.shape[0])

In [10]:
#cost function for the training systems
def func_and_grad(par,system):
    func=0.0
    funcGAGA=0.0
    funcUUCG=0.0
    chi2_native_GAGA=0.0
    chi2_native_UUCG=0.0
    chi2_noe=0.0
    chi2_unoe=0.0
    chi2_backbone1=0.0
    chi2_backbone2=0.0
    chi2_sugar=0.0
    
    GAGAnative=0.0
    UUCGnative=0.0
    
    USE_CUDA=True
    
    if system == 'GAGA':
        funcGAGA=0.0
        gradGAGA=np.zeros(len(par))
        if obs_weight[0] != 0:
            chi2_native_GAGA=0.0
            cm_traj, cm_weights,=activate_cuda(trajGAGA,weightsGAGA, USE_CUDA)
            w=compute_newweights(par,cm_traj,cm_weights,USE_CUDA)

            f=function(par,populationGAGA,w, trajGAGA,cm_traj, USE_CUDA)
            GAGAnative=f[0]
            target_population=0.5
            diff=np.log(f[0]+eps)-np.log(target_population) 
            if(diff<.0):
                    chi2_native_GAGA+=diff**2
            funcGAGA=obs_weight[0]*TLweight*chi2_native_GAGA
            func+=prefactor*(funcGAGA+funcUUCG)
        
    if system == 'UUCG':
    
        funcUUCG=0.0
        gradUUCG=np.zeros(len(par))
        if obs_weight[1] != 0:
            chi2_native_UUCG=0.0
            cm_traj, cm_weights,=activate_cuda(trajUUCG,weightsUUCG, USE_CUDA)
            w=compute_newweights(par,cm_traj,cm_weights,USE_CUDA)

            f=function(par,populationUUCG,w, trajUUCG,cm_traj, USE_CUDA)
            UUCGnative=f[0]
            target_population=0.5
            diff=np.log(f[0]+eps)-np.log(target_population)
            if(diff<.0):
                    chi2_native_UUCG+=diff**2
            funcUUCG=obs_weight[1]*TLweight*chi2_native_UUCG

        func+=prefactor*(funcGAGA+funcUUCG)
    
    if system == 'GACC':
        USE_CUDA=True
        cm_traj, cm_weights,=activate_cuda(trajGACC,weightsGACC, USE_CUDA)
        w=compute_newweights(par,cm_traj,cm_weights,USE_CUDA)

        if obs_weight[2] != 0:
            f=function(par,noe,w, trajGACC,cm_traj, USE_CUDA)
            chi2_noe=np.sum(((f[0]**(-1/6)-noe_exp[:,0])/noe_exp[:,1])**2)/noe_exp.shape[0]
            func+=obs_weight[2]*noe_exp.shape[0]*chi2_noe

        if obs_weight[3] != 0:
            f=function(par,unoe,w, trajGACC,cm_traj, USE_CUDA)
            chi2_unoe=0.0
            for i in range(len(f[0])):
                diff=f[0][i]**(-1/6)-unoe_exp[i,0]
                if(diff<.0):
                    chi2_unoe+=(diff/unoe_exp[i,1])**2/unoe_exp.shape[0]
            func+=obs_weight[3]*unoe_exp.shape[0]*chi2_unoe

        if obs_weight[4] != 0:
            f=function(par,backbone1,w, trajGACC,cm_traj, USE_CUDA)
            chi2_backbone1=np.sum(((f[0]-backbone1_exp[:,0])/backbone1_exp[:,1])**2)/backbone1_exp.shape[0]
            func+=obs_weight[4]*backbone1_exp.shape[0]*chi2_backbone1

        if obs_weight[5] != 0:
            f=function(par,backbone2,w, trajGACC,cm_traj, USE_CUDA)
            chi2_backbone2=np.sum(((f[0]-backbone2_exp[:,0])/backbone2_exp[:,1])**2)/backbone2_exp.shape[0]
            func+=obs_weight[5]*backbone2_exp.shape[0]*chi2_backbone2

        if obs_weight[6] != 0:
            f=function(par,sugar,w, trajGACC,cm_traj, USE_CUDA)
            chi2_sugar=np.sum(((f[0]-sugar_exp[:,0])/sugar_exp[:,1])**2)/sugar_exp.shape[0]
            func+=obs_weight[6]*sugar_exp.shape[0]*chi2_sugar

    func/=TLweight
    
    return float(func),float(GAGAnative),float(UUCGnative)

eps=1e-30

prefactor=1.0
TLweight=(noe_exp.shape[0]+unoe_exp.shape[0] + backbone1_exp.shape[0]+ backbone2_exp.shape[0]+sugar_exp.shape[0])

In [11]:
obs_weight=[1.0,1.0,1.0,1.0,1.0,1.0,1.0]

print('gHBfix19')
lambdas=np.zeros(12)
print('GACC error: ',func_and_grad(lambdas,'GACC')[0])
print('GAGA native population: ',func_and_grad(lambdas,'GAGA')[1]*100)
print('UUCG native population: ',func_and_grad(lambdas,'UUCG')[2]*100)
  
print('gHBfix21 (reweighting)')
lambdas=np.load('./data_produced/parameters.npy',allow_pickle=True)
print('GACC error: ',func_and_grad(lambdas,'GACC')[0])
print('GAGA native population: ',func_and_grad(lambdas,'GAGA')[1]*100)
print('UUCG native population: ',func_and_grad(lambdas,'UUCG')[2]*100)

gHBfix19
GACC error:  0.242701064327384
GAGA native population:  23.6799955368042
UUCG native population:  0.01614885841263458
gHBfix21 (reweighting)
GACC error:  0.3268314550830411
GAGA native population:  65.91648459434509
UUCG native population:  27.310365438461304


In [12]:
#load validation set data
Validation_sequence_list=['CAAU','UUUU','UUCG']

def read_exp(path, columns):
    matrix=[]
    for col in range(columns):
        matrix.append([])
    with open(path, 'r') as f:
        for line in f:
            if '#' not in line:
                nums=line.split()
                for col in range(columns):
                    matrix[col].append(nums[col])
    return np.array(matrix)

def read_UNCG(files):
    corr=[]
    for file in files:
        with open(file, "r") as f:
            for line in f:
                if(line[0]=='#'):
                    continue
                else:
                    nums=line.split()
                    if nums:
                        corr.append(np.array([float(i) for i in nums]))
    return np.array(corr)

backbone1_exp_list=[]
backbone2_exp_list=[]
sugar_exp_list=[]
noe_exp_list=[]
unoe_exp_list=[]

FFprefactorsCAAU=[]
FFprefactorsUUUU=[]
#CAAU parameters
vojtech_dir='/net/sbp/srnas2/vmlynsky/CAAU-for-REWEIGHTING/gHBfix-optimal' 
with open("./data_loaded/CAAU/coefficients.dat","r") as fp:
    for line in fp:
        if(line[0]=='#'):
            continue
        if line.strip():
            l=line.split()
            FFprefactorsCAAU.append(np.array(l[1:],dtype='float'))
#UUUU parameters
with open("./data_loaded/UUUU/coefficients.dat","r") as fp:
    for line in fp:
        if(line[0]=='#'):
            continue
        if line.strip():
            l=line.split()
            FFprefactorsUUUU.append(np.array(l[1:],dtype='float'))
            
backbone1_list=[]
backbone2_list=[]
sugar_list=[]
noe_list=[]
unoe_list=[]
population_list=[]
traj_list=[]
weights_list=[]

for s,Sequence in enumerate(Validation_sequence_list):
    print(Sequence)  
        
    if Sequence == 'UUCG':
        print('There are no experimental population data for the TLs. The population is enforced to be 0.5 in the cost function.')
        
        trajectoryNamesUNCG=[]
        FFprefactorsUNCG=[]
        with open("./data_loaded/UUCG_validation/coefficients_gHBfix21.dat","r") as fp:
            for line in fp:
                if(line[0]=='#'):
                    continue
                if line.strip():
                    l=line.split()
                    FFprefactorsUNCG.append(np.array(l[1:],dtype='float'))
                    file='uucg_ghbfix21_rep0_HBfix-energy_state.dat' 
                    trajectoryNamesUNCG.append('./data_loaded/UUCG_validation/'+file)
                    
        print('Additionally there is the UUCG validation simulation with gHBfix21')
        trajUNCG=read_UNCG([x for x in trajectoryNamesUNCG])
        populationUNCG=trajUNCG[:,-1].reshape(-1,1)
        trajUNCG=trajUNCG[:,:12]

        populationUNCG=populationUNCG[1:]
        trajUNCG=trajUNCG[1:]
        
        print(trajUNCG.shape)
        print(populationUNCG.shape)
    
        traj_list.append(trajUNCG)
        population_list.append(populationUNCG)
        
        Metadweight=[]
        with open("./data_loaded/UUCG_validation/weights.rep0","r") as fp:
            for line in fp:
                Metadweight.append(float(line))
        MetadPot=kBT*np.log(Metadweight)
    
    if Sequence == 'CAAU' or Sequence == 'UUUU':
        j3_data=read_exp('./data_loaded/%s/j3_%s.exp.dat' %(Sequence,Sequence),3 )
        if Sequence=='CAAU':
            unoe_data=read_exp('./data_loaded/%s/formatted_unoe_%s.exp.dat' %(Sequence,Sequence),4 )
        else:
            unoe_data=read_exp('./data_loaded/%s/unoe_%s.exp.dat' %(Sequence,Sequence),4 )
        noe_data=read_exp('./data_loaded/%s/noe_%s.exp.dat' %(Sequence,Sequence),5 )


        print(j3_data.shape)
        print(noe_data.shape)
        print(unoe_data.shape)
        #split j3 into backbone and sugar

        backbone1_exp=[]
        backbone2_exp=[]
        sugar_exp=[]
        for e,elem in enumerate(j3_data.T):
            #print(elem[0])
            if '1H5H4' in elem[0] or '2H5H4'in elem[0]:
                backbone1_exp.append([float(elem[1]),float(elem[2])])
            if '1H5P' in elem[0] or '2H5P' in elem[0] or 'H3P' in elem[0]:
                backbone2_exp.append([float(elem[1]),float(elem[2])])
            if 'H1H2'  in elem[0] or 'H2H3' in elem[0] or 'H3H4' in elem[0]:
                sugar_exp.append([float(elem[1]),float(elem[2])])

        backbone1_exp=np.array(backbone1_exp)
        if Sequence=='CAAU':
            backbone2_exp=np.array(backbone2_exp[:-1])
        backbone2_exp=np.array(backbone2_exp)
        sugar_exp=np.array(sugar_exp)
        print('backbone1: ',backbone1_exp.shape)
        print('backbone2: ',backbone2_exp.shape)
        print('sugar: ',sugar_exp.shape)

        noe_exp=noe_data.T[:,2:]
        unoe_exp=unoe_data.T[:,2:]
        noe_exp=noe_exp[:,:].astype(float)
        noe_exp=np.vstack((noe_exp[:,1],noe_exp[:,2]-noe_exp[:,0])).T
        unoe_exp=unoe_exp[:,:].astype(float)
        print('NOE: ',noe_exp.shape)
        print('uNOE: ',unoe_exp.shape)
        
        backbone1_exp_list.append(backbone1_exp)
        backbone2_exp_list.append(backbone2_exp)
        sugar_exp_list.append(sugar_exp)
        noe_exp_list.append(noe_exp)
        unoe_exp_list.append(unoe_exp)
        
        backbone1=np.load('./data_loaded/%s/backbone1.npy' %(Sequence))
        backbone2=np.load('./data_loaded/%s/backbone2.npy' %(Sequence))
        sugar=np.load('./data_loaded/%s/sugar.npy' %(Sequence))
        noe=np.load('./data_loaded/%s/NOE.npy' %(Sequence))
        unoe=np.load('./data_loaded/%s/uNOE.npy' %(Sequence))
        
        backbone1=9.7*np.cos(backbone1*np.pi/180)**2 - 1.8*np.cos(backbone1*np.pi/180)
        backbone2=15.3*np.cos(backbone2*np.pi/180)**2 - 6.1*np.cos(backbone2*np.pi/180)+1.6
        sugar=9.67*np.cos(sugar*np.pi/180)**2 - 2.03*np.cos(sugar*np.pi/180)
        noe=noe**-6
        unoe=unoe**-6
    
        if Sequence == 'CAAU':
            traj=np.load('./data_loaded/%s/traj.npy' %(Sequence))
        if Sequence == 'UUUU':
            traj=np.load('./data_loaded/%s/traj.npy' %(Sequence))
            traj=np.insert(traj, 1, np.zeros(traj.shape[0]), axis=1)
            traj=np.insert(traj, 7, np.zeros(traj.shape[0]), axis=1)
     
        print('backbone1: ',backbone1.shape)
        print('backbone2: ',backbone2.shape)
        print('sugar: ',sugar.shape)
        print('NOE: ',noe.shape)
        print('uNOE: ',unoe.shape)
        print('corrections: ', traj.shape)


        backbone1_list.append(backbone1)
        backbone2_list.append(backbone2)
        sugar_list.append(sugar)
        noe_list.append(noe)
        unoe_list.append(unoe)
        traj_list.append(traj)
        
    if Sequence == 'CAAU':
        weights=np.exp(bussilab.wham.wham(np.matmul(traj_list[s],np.transpose(FFprefactorsCAAU-FFprefactorsGAGA[1])),threshold=1e-20,T=kBT).logW)
    if Sequence == 'UUUU':
        weights=np.exp(bussilab.wham.wham(np.matmul(traj_list[s],np.transpose(FFprefactorsUUUU-FFprefactorsGAGA[1])),threshold=1e-20,T=kBT).logW)
    if Sequence=='UUCG':
        weights=np.exp(bussilab.wham.wham(np.matmul(traj_list[s],np.transpose(FFprefactorsUNCG-FFprefactorsGAGA[1]))+MetadPot.reshape(-1,1),threshold=1e-20,T=kBT).logW)
    
    print('weights: ',weights.shape)
    weights_list.append(weights)

CAAU
(3, 28)
(5, 39)
(4, 377)
backbone1:  (8, 2)
backbone2:  (8, 2)
sugar:  (11, 2)
NOE:  (39, 2)
uNOE:  (377, 2)
backbone1:  (1000000, 8)
backbone2:  (1000000, 8)
sugar:  (1000000, 11)
NOE:  (1000000, 39)
uNOE:  (1000000, 377)
corrections:  (1000000, 12)
weights:  (1000000,)
UUUU
(3, 25)
(5, 9)
(4, 283)
backbone1:  (6, 2)
backbone2:  (9, 2)
sugar:  (10, 2)
NOE:  (9, 2)
uNOE:  (283, 2)
backbone1:  (1000000, 6)
backbone2:  (1000000, 9)
sugar:  (1000000, 10)
NOE:  (1000000, 9)
uNOE:  (1000000, 283)
corrections:  (1000000, 12)
weights:  (1000000,)
UUCG
There are no experimental population data for the TLs. The population is enforced to be 0.5 in the cost function.
Additionally there is the UUCG validation simulation with gHBfix21
(500000, 12)
(500000, 1)
weights:  (500000,)


In [13]:
#cost function for the validation systems
def cost_func(par,system):
    
    if system == 'CAAU':
            s=0
    if system == 'UUUU':
            s=1
    if system == 'UUCG':
            s=2       
    
    func=0.0
    total_number_observables=0
    if system in ['CAAU', 'UUUU']:
        
        total_number_observables+=obs_weight[0]*backbone1_exp_list[s].shape[0]
        total_number_observables+=obs_weight[1]*backbone2_exp_list[s].shape[0]
        total_number_observables+=obs_weight[2]*sugar_exp_list[s].shape[0]
        total_number_observables+=obs_weight[3]*noe_exp_list[s].shape[0]
        total_number_observables+=obs_weight[4]*unoe_exp_list[s].shape[0]
            
    else:
        total_number_observables+=TLweight
     
    if system in ['CAAU', 'UUUU']: 
        
        UUCGnative=0
            
        weights=weights_list[s]
        traj=traj_list[s]
        
        USE_CUDA=True
        cm_traj, cm_weights,=activate_cuda(traj,weights, USE_CUDA)
        w=compute_newweights(par,cm_traj,cm_weights,USE_CUDA).asarray()   
            
        #---------------------------------------------------------------------------------------
        
        USE_CUDA=True
        cm_traj, cm_weights,=activate_cuda(traj,weights, USE_CUDA)
        w=compute_newweights(par,cm_traj,cm_weights,USE_CUDA)
        
        
        #for the first 3 sequences there are NOE data
        backbone1=backbone1_list[s]
        backbone2=backbone2_list[s]
        sugar=sugar_list[s]

        backbone1_exp=backbone1_exp_list[s]
        backbone2_exp=backbone2_exp_list[s]
        sugar_exp=sugar_exp_list[s]

        f=function(par,backbone1,w, traj,cm_traj, USE_CUDA)
        chi2_backbone1=np.sum(((f[0]-backbone1_exp[:,0])/backbone1_exp[:,1])**2)/backbone1_exp.shape[0]
        func+=obs_weight[0]*backbone1_exp.shape[0]*chi2_backbone1

        f=function(par,backbone2,w, traj,cm_traj, USE_CUDA)
        chi2_backbone2=np.sum(((f[0]-backbone2_exp[:,0])/backbone2_exp[:,1])**2)/backbone2_exp.shape[0]
        func+=obs_weight[1]*backbone2_exp.shape[0]*chi2_backbone2

        f=function(par,sugar,w, traj,cm_traj, USE_CUDA)
        chi2_sugar=np.sum(((f[0]-sugar_exp[:,0])/sugar_exp[:,1])**2)/sugar_exp.shape[0]
        func+=obs_weight[2]*sugar_exp.shape[0]*chi2_sugar

        noe=noe_list[s]
        noe_exp=noe_exp_list[s]
        f=function(par,noe,w, traj,cm_traj, USE_CUDA)
        chi2_noe=np.sum(((f[0]**(-1/6)-noe_exp[:,0])/noe_exp[:,1])**2)/noe_exp.shape[0]
        func+=obs_weight[3]*noe_exp.shape[0]*chi2_noe

        unoe=unoe_list[s]
        unoe_exp=unoe_exp_list[s]
        f=function(par,unoe,w, traj,cm_traj, USE_CUDA)
        chi2_unoe=0.0
        grad_unoe=np.zeros(len(par))
        for i in range(len(f[0])):
            diff=f[0][i]**(-1/6)-unoe_exp[i,0]
            if(diff<.0):
                chi2_unoe+=(diff/unoe_exp[i,1])**2/unoe_exp.shape[0]
        func+=obs_weight[4]*unoe_exp.shape[0]*chi2_unoe
                
    else:
        weights=weights_list[s]
        traj=traj_list[s]
        
        USE_CUDA=True
        cm_traj, cm_weights,=activate_cuda(traj,weights, USE_CUDA)
        w=compute_newweights(par,cm_traj,cm_weights,USE_CUDA).asarray()   

        population=np.array(population_list[0])

        chi2_native_pop=0.0
        grad_native_pop=np.zeros(len(par))
        f=function(par,population,w, traj,cm_traj, USE_CUDA)
        UUCGnative=f[0]
        target_population=0.5
        diff=np.log(f[0]+eps)-np.log(target_population) #i want to penalize diff < 0, because large and negative means non-native
        if(diff<.0):
                chi2_native_pop+=diff**2
        func+=obs_weight[5]*TLweight*chi2_native_pop

        
    func/=total_number_observables
    
    return float(func),float(UUCGnative)

In [14]:
obs_weight=[1.0,1.0,1.0,1.0,1.0,1.0,1.0]

print('gHBfix19 (reweighting)')
lambdas=np.zeros(12)
print('CAAU error: ',cost_func(lambdas,'CAAU')[0])
print('UUUU error: ',cost_func(lambdas,'UUUU')[0])
print('UUCG native population: ',cost_func(lambdas,'UUCG')[1]*100)
  
print('gHBfix21')
lambdas=np.load('./data_produced/parameters.npy',allow_pickle=True)
print('CAAU error: ',cost_func(lambdas,'CAAU')[0])
print('UUUU error: ',cost_func(lambdas,'UUUU')[0])
print('UUCG native population: ',cost_func(lambdas,'UUCG')[1]*100)

gHBfix19 (reweighting)
CAAU error:  5.00054139785316
UUUU error:  1.7308205221083381
UUCG native population:  0.0028775624741683714
gHBfix21
CAAU error:  3.6371643729773138
UUUU error:  1.4808770637285222
UUCG native population:  20.76646089553833
